In [2]:
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.utils import hermitian_conjugated
from openfermion.utils import uccsd_generator
import numpy as np

from pyquil.paulis import PauliSum
from pyquil.api import WavefunctionSimulator
from scipy.optimize import minimize
from pyquil import Program
from pyquil.gates import *

from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli
from forestopenfermion import exponentiate

import numpy as np
import functools

from openfermion.hamiltonians import MolecularData, load_molecular_hamiltonian

In [3]:
sim = WavefunctionSimulator()
numQubit = 4

def solve_vqe(hamiltonian: PauliSum, numQubit, mode) -> float:
    # Construct a variational quantum eigensolver solution to find the lowest
    # eigenvalue of the given hamiltonian
    
    # mode 0: ONLY 'spin-physical' terms (n-1 parameters)
    # mode 1: ALL possible terms (n(n_1)/2 + n(n_1)(n-2)(n-3)/4 parameters)
    
    numParams = 0
    if mode == 0:
        numParams = int((numQubit/2)*(numQubit/2-1) + (numQubit/2)*(numQubit/2-1)/2*(numQubit/2)*(numQubit/2-1)/2)
    elif mode == 1:
        numParams = int(numQubit*(numQubit-1)/2 + numQubit*(numQubit-1)*(numQubit-2)*(numQubit-3)/4)
    params_init = np.random.rand(numParams)*0.5-0.25
    
    def ansatz_energy(params):
        p = None
        p = Program()
        
        # Reference state
        p += X(0)
        
        single_amp = []
        double_amp = []
        # Only 'spin-physical' terms in UCC
        if mode == 0:
            count = 0
            for i in range(int(numQubit/2)-1):
                for j in range(i+1, int(numQubit/2)):
                    single_amp.append([[2*i,2*j], params[count]])
                    count += 1
                    single_amp.append([[2*i+1, 2*j+1], params[count]])
                    count += 1
                    
            for i in range(int(numQubit/2)-1):
                for j in range(int(numQubit/2)-1):
                    for ii in range(i+1, int(numQubit/2)):
                        for jj in range(j+1, int(numQubit/2)):
                            double_amp.append([[2*i,2*j+1,2*ii,2*jj+1], params[count]])
                            count += 1
        
        # Every possible terms
        elif mode == 1:
            count = 0
            for i in range(numQubit-1):
                for j in range(i+1, numQubit):
                    single_amp.append([[i,j], params[count]])
                    count += 1

            for i in range(numQubit-1):
                for j in range(i+1, numQubit):
                    indices = np.arange(numQubit)
                    rem = np.setdiff1d(indices, [i, j])
                    for ii in range(len(rem)-1):
                        for jj in range(ii+1, len(rem)):
                            double_amp.append([[i,j,rem[ii],rem[jj]], params[count]])
                    count += 1
        
        ucc_gen = uccsd_generator(single_amp, double_amp)
        ucc_program = exponentiate(bravyi_kitaev(ucc_gen)/(-1j))
        p += ucc_program

        energy = sim.expectation(p, hamiltonian).real
        return energy

    option = {}
    option['maxiter']=10000
    option['disp']=True
    params_answer = minimize(ansatz_energy, params_init, method='L-BFGS-B', options=option).x
    #print(params_answer)
    return ansatz_energy(params_answer)

In [4]:
def get_ground_energy(interaction_hamil, numQubit, mode):
    fermionop_hamil = FermionOperator()
    for key in interaction_hamil:
        value = interaction_hamil[key]
        fermionop_hamil += FermionOperator(term=key, coefficient=value)
        
    qubitop_hamil = bravyi_kitaev(fermionop_hamil)
    pauliop_hamil = qubitop_to_pyquilpauli(qubitop_hamil)
    
    sim = WavefunctionSimulator(random_seed=1337)
    return solve_vqe(pauliop_hamil, numQubit, mode)


In [5]:
basis = 'sto-3g'
multiplicity = 1  # 2S+1
charge = 0

import matplotlib.pyplot as plt
%matplotlib inline

bond_lengths = np.linspace(0.3, 2.5, 23)
vqe_ground_energies0 = []
vqe_ground_energies1 = []

for bond_length in bond_lengths:
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    description = str(round(bond_length, 2))
    h2_interaction_hamil = load_molecular_hamiltonian(geometry,
        basis,
        multiplicity,
        description,
        n_active_electrons=None,
        n_active_orbitals=None)
    
    ge = get_ground_energy(h2_interaction_hamil, 4, 0)
    vqe_ground_energies0.append(ge)
    print('bond length: ', round(bond_length, 2), ' ground state energy: ', ge)
    ge = get_ground_energy(h2_interaction_hamil, 4, 1)
    vqe_ground_energies1.append(ge)
    print('bond length: ', round(bond_length, 2), ' ground state energy: ', ge)


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /qvm (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1324ff9390>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [ ]:
# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
bond_length_interval = 0.1
n_points = 25

# Generate molecule at different bond lengths.
hf_energies = []
fci_energies = []
bond_lengths = []
for point in range(3, n_points + 1):
    bond_length = bond_length_interval * point
    bond_lengths += [bond_length]
    description = str(round(bond_length,2))
#    print(description)
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    molecule = MolecularData(
        geometry, basis, multiplicity, description=description)
    
    # Load data.
    molecule.load()

    # Print out some results of calculation.
#    print('\nAt bond length of {} angstrom, molecular hydrogen has:'.format(
#        bond_length))
#    print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
#    print('MP2 energy of {} Hartree.'.format(molecule.mp2_energy))
#    print('FCI energy of {} Hartree.'.format(molecule.fci_energy))
#    print('Nuclear repulsion energy between protons is {} Hartree.'.format(
#        molecule.nuclear_repulsion))
#    for orbital in range(molecule.n_orbitals):
#        print('Spatial orbital {} has energy of {} Hartree.'.format(
#            orbital, molecule.orbital_energies[orbital]))
    hf_energies += [molecule.hf_energy]
    fci_energies += [molecule.fci_energy]

plt.figure(0)
plt.plot(bond_lengths, fci_energies, 'o-')
plt.plot(bond_lengths, hf_energies, '+-')
plt.plot(bond_lengths, vqe_ground_energies0, 'x-')
plt.plot(bond_lengths, vqe_ground_energies1, 'x-')
plt.ylabel('Energy in Hartree')
plt.xlabel('Bond length in angstrom')
plt.show()

print(fci_energies)
print(vqe_ground_energies0)
print(vqe_ground_energies1)